<a href="https://colab.research.google.com/github/azizamirsaidova/Seq2Seq-Polynomials-/blob/main/Seq2Seq_Polymonial_Expansion_with_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Deep learning model that learns to expand single variable polynomials by taking the factorized sequence and predicting the expanded sequence.**








In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

In [ ]:
data = pd.read_csv('train.txt', sep=" ", header=None)
data[['cleaned_source', 'cleaned_target']] = data[0].str.split('=', 1, expand=True)
data['cleaned_target'] = data['cleaned_target'].apply(lambda x: "\t" + x + "\n")

In [ ]:
data['cleaned_source'].head()

0    (7-3*z)*(-5*z-9)
1             -9*s**2
2       (2-2*n)*(n-1)
3                x**2
4        (4-x)*(x-23)
Name: cleaned_source, dtype: object

In [ ]:
data['cleaned_target'].head()

0    \t15*z**2-8*z-63\n
1           \t-9*s**2\n
2     \t-2*n**2+4*n-2\n
3              \tx**2\n
4     \t-x**2+27*x-92\n
Name: cleaned_target, dtype: object

In [ ]:
with open('train.txt', "r", encoding="utf-8") as f:
    lines = f.read().split("\n")

In [ ]:
lines = lines[0:100000]

In [ ]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

for line in lines:
    if line != '':
      input_text, target_text = line.split("=")
      # for the targets, and "\n" as "end sequence" character.
      target_text = "\t" + target_text + "\n"
      input_texts.append(input_text)
      target_texts.append(target_text)
      for char in input_text:
          if char not in input_characters:
              input_characters.add(char)
      for char in target_text:
          if char not in target_characters:
              target_characters.add(char)

In [ ]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 100000
Number of unique input tokens: 28
Number of unique output tokens: 30
Max sequence length for inputs: 29
Max sequence length for outputs: 30


In [ ]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
input_token_index[" "] = len(input_token_index)
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])
target_token_index[" "] = len(target_token_index)

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    # encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    # decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    # decoder_target_data[i, t:, target_token_index[" "]] = 1.0


In [ ]:
decoder_target_data[0][0]

array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [ ]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.

# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 28)]   0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None, 30)]   0           []                               
                                                                                                  
 lstm (LSTM)                    [(None, 256),        291840      ['input_1[0][0]']                
                                 (None, 256),                                                     
                                 (None, 256)]                                                     
                                                                                              

In [ ]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=5,
    validation_split=0.2,
)
# Save model
model.save("s2s")


Epoch 1/5
1250/1250 [==============================] - 49s 34ms/step - loss: 0.9834 - accuracy: 0.2264 - val_loss: 0.7720 - val_accuracy: 0.2754
Epoch 2/5
1250/1250 [==============================] - 40s 32ms/step - loss: 0.7155 - accuracy: 0.2991 - val_loss: 0.6539 - val_accuracy: 0.3172
Epoch 3/5
1250/1250 [==============================] - 39s 31ms/step - loss: 0.5823 - accuracy: 0.3467 - val_loss: 0.5338 - val_accuracy: 0.3631
Epoch 4/5
1250/1250 [==============================] - 41s 33ms/step - loss: 0.5094 - accuracy: 0.3738 - val_loss: 0.4694 - val_accuracy: 0.3891
Epoch 5/5
1250/1250 [==============================] - 40s 32ms/step - loss: 0.4583 - accuracy: 0.3936 - val_loss: 0.4321 - val_accuracy: 0.3990


INFO:tensorflow:Assets written to: s2s/assets


INFO:tensorflow:Assets written to: s2s/assets


In [ ]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2s")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence



In [ ]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)
    print("Target sentence:", target_texts[seq_index][1:-1])


-
Input sentence: (7-3*z)*(-5*z-9)
Decoded sentence: 15*z**2-11*z-10

Target sentence: 15*z**2-8*z-63
-
Input sentence: -9*s**2
Decoded sentence: -9*s**2

Target sentence: -9*s**2
-
Input sentence: (2-2*n)*(n-1)
Decoded sentence: -2*n**2+13*n-10

Target sentence: -2*n**2+4*n-2
-
Input sentence: x**2
Decoded sentence: x**2

Target sentence: x**2
-
Input sentence: (4-x)*(x-23)
Decoded sentence: -x**2+23*x-10

Target sentence: -x**2+27*x-92
-
Input sentence: (7-5*c)*(3*c-17)
Decoded sentence: -15*c**2+115*c-10

Target sentence: -15*c**2+106*c-119
-
Input sentence: -8*x*(3*x+14)
Decoded sentence: -24*x**2-100*x

Target sentence: -24*x**2-112*x
-
Input sentence: -2*k*(5*k-9)
Decoded sentence: -10*k**2+20*k

Target sentence: -10*k**2+18*k
-
Input sentence: (3*cos(c)-19)*(7*cos(c)+13)
Decoded sentence: 15*cos(n)**2+126*cos(n)-1

Target sentence: 21*cos(c)**2-94*cos(c)-247
-
Input sentence: -8*j*(-8*j-3)
Decoded sentence: 48*j**2+16*j

Target sentence: 64*j**2+24*j
-
Input sentence: (h+8)*(7*h